# 進化的モデルマージでAIモデルをパワーアップ！mergekit-evolve徹底解説

**進化的なモデルマージ**とは、複数の言語モデルを組み合わせて、特定の能力や特性を持つ新しいモデルを作成する手法です。従来の手法では、どのモデルをどのように組み合わせるかは手探りで進める必要がありましたが、進化的なモデルマージでは、目標とする特性を指定することで、最適な組み合わせを自動的に探索できます。

この記事では、進化的なモデルマージを誰でも手軽に試せるツール「mergekit-evolve」の使い方を、初心者にもわかりやすく解説します。基本的なインストールから、モデルの評価、そして最終的なモデルの作成まで、順を追って説明していきます。コード例も豊富に用意し、実際に手を動かしながら理解を深められるように工夫しました。

## 1. 準備：必要な環境を整えよう

進化的なモデルマージを行うには、GPUを搭載した環境が必要です。7B程度のモデルであれば、24GBのVRAMがあれば十分です。

### mergekitのインストール

まずは、mergekitをインストールしましょう。以下のコマンドをターミナルで実行します。

In [2]:
!pip install --upgrade pip
!git clone -b develop https://github.com/Sunwood-ai-labs/EMEX.git
%cd EMEX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 33.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Cloning into 'EMEX'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 65 (delta 24), reused 60 (delta 19), pack-reused 0
Receiving objects: 100% (65/65), 15.91 KiB | 7.95 MiB/s, done.
Resolving deltas: 100% (24/24), done.
/content/EMEX


In [3]:
!rm -rf /opt/conda/lib/python3.10/site-packages/aiohttp*
!rm -rf /opt/conda/lib/python3.10/site-packages/multidict*

In [4]:
!pip install git+https://github.com/arcee-ai/mergekit.git#egg=mergekit[evolve,vllm]

DEPRECATION: git+https://github.com/arcee-ai/mergekit.git#egg=mergekit[evolve,vllm] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Cloning https://github.com/arcee-ai/mergekit.git to /tmp/pip-install-tprf8_zm/mergekit_f5407e02388948ebb15e01c534a2f7d4
  Running command git clone --filter=blob:none --quiet https://github.com/arcee-ai/mergekit.git /tmp/pip-install-tprf8_zm/mergekit_f5407e02388948ebb15e01c534a2f7d4
  Resolved https://github.com/arcee-ai/mergekit.git to commit 107008ead4ae3ff3aa0671bcc79feda20807f139
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 4.0 MB/s eta 0:00:00
  Preparing metadata 

In [5]:
username = 'MakiAi'
token = 'HF_TOKEN'
license = "apache-2.0"

!pip install -qU huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 10.0 MB/s eta 0:00:00


In [6]:
import yaml

from huggingface_hub import ModelCard, ModelCardData, HfApi

try:
    from google.colab import userdata
    HF_TOKEN = userdata.get(token)
    WANDB_API_KEY = userdata.get('WANDB_API_KEY')
except:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    HF_TOKEN = user_secrets.get_secret(token)
    WANDB_API_KEY = user_secrets.get_secret('WANDB_API_KEY')


In [7]:
from huggingface_hub import login
login(token=HF_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## 2. タスクの定義：モデルに何をさせたいか決めよう

進化的なモデルマージでは、モデルの評価基準となるタスクを定義する必要があります。ここでは、空間認識能力を評価する`spartqa-mchoice`と、プロンプト形式の応答能力を評価する`alpaca-gpt4`の2つのタスクを定義します。

### spartqa-mchoiceデータセットの準備


In [8]:
import datasets
# Hugging Face Datasetsからspartqa-mchoiceデータセットをロード。["train"]は訓練データのみを取得するという意味
ds = datasets.load_dataset("metaeval/spartqa-mchoice")["train"]
# データをシャッフルし、ランダムに1000個のサンプルを選択
ds_p = ds.shuffle(seed=9163).select(range(1000))
# 選択したサンプルをHugging Face Hubにアップロード(private=Trueは非公開でアップロードするという意味)
ds_p.push_to_hub(f"{username}/spartqa-train-1k", private=True, token=HF_TOKEN)


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/414 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/MakiAi/spartqa-train-1k/commit/7d20b5aff6853cc57f3679c97d5ca631f54f8f9f', commit_message='Upload dataset', commit_description='', oid='7d20b5aff6853cc57f3679c97d5ca631f54f8f9f', pr_url=None, pr_revision=None, pr_num=None)

### alpaca-gpt4データセットの準備とタスク定義

`spartqa-mchoice`と同様の手順で、`alpaca-gpt4`データセットの準備とタスク定義を行います。

# Hugging Face Datasetsからalpaca-gpt4データセットをロード。["train"]は訓練データのみを取得するという意味

In [9]:
ds = datasets.load_dataset("vicgalle/alpaca-gpt4")["train"]
df = ds.to_pandas() # pandasのデータフレームに変換

no_input = df[df.input.map(len) < 1]
examples = no_input.sample(n=500, replace=False, random_state=749)
ds_p = datasets.Dataset.from_pandas(examples)
ds_p.push_to_hub(f"{username}/alpaca-gpt4-500", private=True, token=HF_TOKEN) # 選択したサンプルをHugging Face Hubにアップロード(private=Trueは非公開でアップロードするという意味)


Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/426 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/MakiAi/alpaca-gpt4-500/commit/e43f7c296cf9a29dd201699b2a4586164e35836a', commit_message='Upload dataset', commit_description='', oid='e43f7c296cf9a29dd201699b2a4586164e35836a', pr_url=None, pr_revision=None, pr_num=None)

## 4. マージの実行と最終モデルの取得

いよいよ、進化的なモデルマージを実行します。

```bash
# 進化的なモデルマージを実行
!mergekit-evolve ./evol_merge_config.yml \  
    # マージ結果を保存するディレクトリを指定 \
    --storage-path /workspace/evol_merge_storage \  
    # タスク定義ファイルを保存しているディレクトリを指定 \
    --task-search-path /workspace/eval_tasks \  
    # vllmバックエンドを使用 (vllmはモデルの推論を高速化するためのライブラリ)
    --vllm \  
    # モデルをメモリにロード (高速化のため)
    --in-memory \  
    # GPUを使用してマージ (高速化のため)
    --merge-cuda \  
    # Weights & Biasesにログを送信 (実験の進捗状況や結果を記録・可視化できるツール)
    --wandb  

```

In [10]:
!ls

eval_tasks  evol_merge_config.yml  README.md  script


In [11]:
!ls eval_tasks

alpaca_prompt_format.yaml  preprocess_spartqa.py  spartqa_1k_train.yaml


In [12]:
!pwd

/content/EMEX


In [13]:
!wandb login $WANDB_API_KEY

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# 進化的なモデルマージを実行
!mergekit-evolve ./evol_merge_config.yml \
    --storage-path /workspace/evol_merge_storage \
    --task-search-path /workspace/eval_tasks \
    --vllm \
    --in-memory \
    --merge-cuda \
    --wandb

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
2024-06-05 14:37:04.365556: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-05 14:37:04.419638: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-05 14:37:04.419691: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-05 14:37:04.421655: E exte

# 最適なモデルの設定を読み込んで、最終的なモデルを作成 (--cudaはGPUを使用するオプション)
マージが完了すると、最適なモデルの設定が`/workspace/evol_merge_storage/best_config.yaml`に保存されます。

In [ ]:
!mergekit-yaml /workspace/evol_merge_storage/best_config.yaml --cuda /workspace/final_merge


これで、進化的なモデルマージによって作成された、あなたの目的に合った新しいモデルが完成しました！